In [1]:
import pandas as pd
from io import StringIO
import numpy as np
#import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from pylab import rcParams
rcParams['figure.figsize'] = 15, 15
import wellapplication as wa
import matplotlib as mpl
import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
import arcpy
arcpy.CheckOutExtension("spatial")
from arcpy import env 
from arcpy.sa import *

In [3]:
def getdict(wrksp, yrstrt = -6):
    if yrstrt >= -4:
        yrend = None
    else:
        yrend = yrstrt+4
    arcpy.env.workspace = wrksp
    dct = {}
    for tab in arcpy.ListTables():
        if tab[yrstrt:yrend] in [str(i) for i in range(2000,2018)]:

            dct[tab.replace('zonal_','')] = table_to_pandas_dataframe(tab)
    return dct

def dictozone(dct, yrstrt = -6, dataend = 4, div = 1.0):
    if yrstrt >= -4:
        yrend = None
        moend = None
    elif yrstrt <= -6:
        yrend = yrstrt + 4
        moend = yrstrt + 6
    else:
        yrend = yrstrt+4
        moend = None
    zonal = pd.concat(dct)
    zonal['Year'] = zonal.index.get_level_values(0).map(lambda x: x[yrstrt:yrend])
    zonal['Month'] = zonal.index.get_level_values(0).map(lambda x: x[yrend:moend])
    zonal['Dataset'] = zonal.index.get_level_values(0).map(lambda x: x[:dataend])
    zonal['volume_acft'] = zonal[['AREA','MEAN']].apply(lambda x: x[0]*x[1]*0.000810714/div,1)
    zonalyr = zonal.groupby(['Dataset','Year']).sum()
    zonal.reset_index(inplace=True)
    return zonal

def zonal_stack(zonal, zoneid = 'Permanent_Identifier', dmin=2004,dmax=2014):
    znl = zonal.drop(['level_0','level_1','ZONE_CODE','COUNT','AREA','MEAN'],axis=1)
    zstack = znl.groupby(['Dataset', zoneid,'Year']).sum()
    ztab = zstack.unstack('Year')['volume_acft']
    drange = [str(i) for i in range(dmin,dmax+1)]
    print(ztab.columns)
    ztab['average'] = ztab[drange].apply(lambda x: np.mean(x),1)
    ztab['min'] = ztab[drange].apply(lambda x: np.min(x),1)
    ztab['max'] = ztab[drange].apply(lambda x: np.max(x),1)
    return ztab

def get_sum_rows(ztab, zoneid = 'Permanent_Identifier'):
    zsum = ztab.groupby(['Dataset',zoneid]).sum()
    sums = zsum.groupby(zsum.index.get_level_values(0)).sum()
    sums[zoneid] = 'All'
    zsum.reset_index(inplace=True)
    sums.reset_index(inplace=True)
    zsum = zsum.append(sums)
    zsum.sort_values(['Dataset',zoneid],ascending=[True,False],inplace=True)
    zsum.set_index(['Dataset',zoneid],inplace=True)
    zsum = zsum.round(0)
    return zsum

In [4]:
def get_field_names(table):
    read_descr = arcpy.Describe(table)
    field_names = []
    for field in read_descr.fields:
        field_names.append(field.name)
    field_names.remove('OBJECTID')
    return field_names


def table_to_pandas_dataframe(table, field_names=None, query=None, sql_sn=(None, None)):
    """
    Load data into a Pandas Data Frame for subsequent analysis.
    :param table: Table readable by ArcGIS.
    :param field_names: List of fields.
    :return: Pandas DataFrame object.
    """
    # if field names are not specified
    if not field_names:
        field_names = get_field_names(table)
    # create a pandas data frame
    df = pd.DataFrame(columns=field_names)

    # use a search cursor to iterate rows
    with arcpy.da.SearchCursor(table, field_names, query, sql_clause=sql_sn) as search_cursor:
        # iterate the rows
        for row in search_cursor:
            # combine the field names and row items together, and append them
            df = df.append(dict(zip(field_names, row)), ignore_index=True)

    # return the pandas data frame
    return df

In [5]:
processed_dir = 'E:\PROJECTS\Round_Valley\SWAT\Scenarios\Default\TxtInOut'

# Read Output Files

##  ArcSWAT Monthly Summary

In [ ]:
 
mondata =  """1	76.66	62.80	2.24	5.73	10.21	1.52	0.23	3.94
2	67.52	51.10	4.23	11.79	19.06	2.76	0.84	6.39
3	51.06	29.79	12.66	35.51	54.98	14.77	2.84	29.59
4	61.93	18.58	15.99	48.01	78.84	27.99	5.18	57.68
5	49.76	1.97	15.77	36.50	75.27	45.14	7.33	117.18
6	25.69	0.00	2.72	7.27	30.00	46.12	2.46	183.36
7	22.11	0.00	0.01	2.86	13.15	44.94	0.01	212.47
8	30.99	0.00	0.27	4.38	7.84	36.81	0.25	181.54
9	48.87	0.68	0.93	7.90	9.70	31.14	0.56	125.19
10	57.20	5.06	1.96	11.05	13.54	26.38	0.46	69.69
11	47.99	27.66	0.63	6.62	8.16	12.43	0.11	24.40
12	94.56	75.62	3.45	6.82	11.73	2.97	0.93	5.00"""

In [ ]:
mondata2 =  """1	91.08	80.27	0.39	4.30	6.12	1.42	0.03	3.54
2	77.13	39.11	4.15	32.39	41.01	4.19	0.52	8.62
3	50.23	20.56	3.28	43.55	59.18	18.52	0.79	38.19
4	48.25	10.60	1.98	39.85	61.25	30.97	0.65	70.44
5	50.39	2.38	3.55	24.95	51.08	49.97	2.04	108.81
6	13.77	0.00	0.00	2.02	18.14	46.40	0.00	199.05
7	21.01	0.00	0.00	2.66	8.43	44.35	0.00	217.53
8	41.22	0.00	0.02	6.61	7.96	39.17	0.00	176.75
9	68.58	1.33	0.45	12.37	13.38	35.99	0.21	131.06
10	22.47	0.08	0.00	3.57	4.33	28.68	0.00	77.86
11	47.21	27.98	0.02	5.54	6.18	11.88	0.00	25.81
12	90.03	68.86	0.69	7.70	9.09	3.76	0.03	6.38"""

In [ ]:
cols = ['mon','rain','snofall','surfq','latq','wtryld','et','sedyld','PET']
monthly_basin_values = pd.read_table(StringIO(mondata2),sep='\t',names=cols)
basinareakm = 181.49 #186.37
km2_to_m2 = 1000000
basinaream = basinareakm*km2_to_m2
for col in cols[1:]:
    monthly_basin_values[col] = monthly_basin_values[col].apply(lambda x: x/1000*basinaream*0.000810714,1)

monthly_basin_values.set_index('mon',inplace=True)
monthly_basin_values.loc['total',:] = monthly_basin_values.loc[1:12,:].sum(axis=0)
monthly_basin_values.loc[:,'avail_water'] = monthly_basin_values.loc[:,'rain'] - monthly_basin_values.loc[:,'et'] 
#monthly_basin_values.round(0).to_clipboard()
monthly_basin_values.round(0)

In [ ]:
cols = ['mon','rain','snofall','surfq','latq','wtryld','et','sedyld','PET']
monthly_basin_values = pd.read_table(StringIO(mondata),sep='\t',names=cols)
basinareakm = 181.49 #186.37
km2_to_m2 = 1000000
basinaream = basinareakm*km2_to_m2
for col in cols[1:]:
    monthly_basin_values[col] = monthly_basin_values[col].apply(lambda x: x/1000*basinaream*0.000810714,1)

monthly_basin_values.set_index('mon',inplace=True)
monthly_basin_values.loc['total',:] = monthly_basin_values.loc[1:12,:].sum(axis=0)
monthly_basin_values.loc[:,'avail_water'] = monthly_basin_values.loc[:,'rain'] - monthly_basin_values.loc[:,'et'] 
#monthly_basin_values.round(0).to_clipboard()
monthly_basin_values.round(0)


## ArcSWAT Model Summary

In [ ]:
yrcols = ['et','ppt','revap','perc','latq','surfq','returnq','rchrg']
yrdata  = [304.3,777.2,81.87,171.29,257.03,28.82,82.21,8.56]

for d in range(len(yrdata)):
    convd = yrdata[d]/1000*basinaream*0.000810714
    print('{:} = {:0.0f} acft/yr'.format(yrcols[d],convd))
    

In [ ]:
yrly['totalppt'] = yrly['rain'] + yrly['snofall']

In [ ]:
yrly['totalppt'] - yrly['et']

In [ ]:
yrly['surfq'] + yrly['latq'] + yrly['wtryld']

## ArcSWAT TxtInOut RCH Summary for Monthly Output

In [ ]:
output_dir = 'E:/Google Drive/WORK/Round_Valley/Data/GIS/SWAT/Scenarios/short/TxtInOut/'

Long_Run_Mar_2018

In [ ]:
cols = ['RCH', 'GIS', 'MON', 'AREAkm2', 'FLOW_INcms', 'FLOW_OUTcms', 'EVAPcms', 'TLOSScms']
rch = pd.read_table(output_dir+'output.rch',sep='\s+',usecols=[1,2,3,4,5,6,7,8],skiprows=9,names=cols)
rch['FLOW_OUTacftyr'] = rch['FLOW_OUTcms']*25566.7
rch.groupby('RCH').median()

# ArcSWAT TxtInOut HRU Summary for Daily Output

In [ ]:
output_dir = 'F:/GIS/Ogden_Valley/SWAT_2017_09_11/Scenarios/Model_w_SWATCUP_params_12_2017/TxtInOut/'
processed_dir = 'F:/GIS/Ogden_Valley/SWAT_2017_09_11/Scenarios/'

In [ ]:
names = ['LULC', 'HRU', 'GIS', 'SUB', 'MGT', 'MO', 'DA', 'YR', 'AREAkm2',
       'PRECIPmm', 'SNOFALLmm', 'SNOMELTmm', 'IRRmm', 'PETmm', 'ETmm',
       'SW_INITmm', 'SW_ENDmm', 'PERCmm', 'GW_RCHGmm', 'DA_RCHGmm', 'REVAPmm',
       'SA_IRRmm', 'DA_IRRmm', 'SA_STmm', 'DA_STmm','SURQ_GENmm','SURQ_CNTmm',
       'TLOSSmm', 'LATQGENmm', 'GW_Qmm', 'WYLDmm']

hru = pd.read_table(output_dir+'output.hru', sep='\s+', usecols=range(0,31,1),
                    skiprows=9, names=names)

In [ ]:
areas = hru.drop_duplicates(['GIS','AREAkm2'])

In [ ]:
areas = areas[['LULC','HRU','GIS','SUB','AREAkm2']]
areas.to_csv(processed_dir+'hru_area_SWT_CUP.csv')

In [ ]:
hru_points = pd.read_csv("F:/GIS/Ogden_Valley/SWAT_2017_09_11/hru_pnts_coords3.txt")

In [ ]:
arcpy.env.overwriteOutput = True
# Set environment settings
arcpy.env.workspace = "F:/GIS/Ogden_Valley/SWAT_2017_09_11/out_Data.gdb"

### Monthly Mean

In [ ]:
hru_mo_mean = hru.groupby(['GIS','YR','MO']).sum()
hru_mo_mean['HRUGIS'] = hru_mo_mean.index.get_level_values(0)
hru_mo_mean['HRUGIS'] = hru_mo_mean['HRUGIS'].apply(lambda x: str(x).zfill(9),1)
hru_mo_mean.to_csv(processed_dir+'hru_mo_mean.csv')

In [ ]:
hru_mo_mean_all = hru_mo_mean.groupby([hru_mo_mean.index.get_level_values(0),hru_mo_mean.index.get_level_values(2)]).mean()
hru_mo_mean_all.to_csv(processed_dir + 'hru_mo_mean_all.csv')

### Yearly Mean

In [ ]:
hru_yr_mean = hru.groupby(['GIS','YR']).sum()
hru_yr_mean['HRUGIS'] = hru_yr_mean.index.get_level_values(0)
hru_yr_mean['HRUGIS'] = hru_yr_mean['HRUGIS'].apply(lambda x: str(x).zfill(9),1)
hru_yr_mean.to_csv(processed_dir+'hru_yr_mean.csv')

In [ ]:
hru_points

In [ ]:
hru_yr_mean.drop(['MO','DA','AREAkm2'],axis=1,inplace=True)

In [ ]:
hru_yr_mean.reset_index(inplace=True)

In [ ]:
hru_points_yrly = pd.merge(hru_yr_mean, hru_points, left_on='GIS', right_on='HRUGIS', how='left')

In [ ]:
hru_points_yrly.to_csv(processed_dir+"hru_points_yrly_2.csv")

In [ ]:
arcpy.env.overwriteOutput = True
# Set environment settings
arcpy.env.workspace = "F:/GIS/Ogden_Valley/SWAT_2017_09_11/out_Data.gdb"

dataset = 'hru_yr_points2'


intfields = ['PRECIPmm','SNOFALLmm','SNOMELTmm','IRRmm','PETmm',
             'ETmm','SW_INITmm','SW_ENDmm','PERCmm','GW_RCHGmm','DA_RCHGmm',
             'REVAPmm','SA_IRRmm','SA_STmm','DA_STmm',
             'SURQ_GENmm','SURQ_CNTmm','TLOSSmm','LATQGENmm','GW_Qmm','WYLDmm']



cellSize = 150

arcpy.env.mask = "F:/GIS/Ogden_Valley/SWAT_2017_09_11/out_Data.gdb/Basin"
# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")



for YR in range(2010,2017):

    # Make a layer from the feature class
    arcpy.MakeFeatureLayer_management(dataset, "lyr") 
 
    # Within selected features, further select only those cities which have a population > 10,000   
    arcpy.SelectLayerByAttribute_management("lyr", "SUBSET_SELECTION", ' "YR" = {:}'.format(YR))
 
    # Write the selected features to a new featureclass
    #arcpy.CopyFeatures_management("lyr", "chihuahua_10000plus")
    
    for zField in intfields:
        # Execute NaturalNeighbor
        outNatNbr = NaturalNeighbor("lyr", zField, cellSize)
    
        # Save the output 
        outNatNbr.save(zField+str(YR))
        print(zField+str(YR))

In [ ]:
arcpy.env.overwriteOutput = True
# Set environment settings
arcpy.env.workspace = "F:/GIS/Ogden_Valley/SWAT_2017_09_11/out_Data.gdb"

# Set local variables
inZoneData = "watersheds_8"
zoneField = "Permanent_Identifier"

for rast in arcpy.ListRasters():
    inValueRaster = rast
    outTable = "zonal_{:}".format(rast)
    try:
        outZSaT = ZonalStatisticsAsTable(inZoneData, zoneField, inValueRaster, outTable, "DATA", "MEAN")
    except:
        pass

In [ ]:
swtdct = getdict("F:/GIS/Ogden_Valley/SWAT_2017_09_11/out_Data.gdb",yrstrt = -4)
swtzone = dictozone(swtdct, yrstrt = -4, dataend = -4, div=1000.0)
swttab = zonal_stack(swtzone, zoneid = 'Permanent_Identifier', dmin=2010, dmax=2016)
SWTdata = get_sum_rows(swttab)

### All Years

In [ ]:
hru_yr_mean_all = hru_yr_mean.groupby([hru_yr_mean.index.get_level_values(0)]).mean()
hru_yr_mean_all['HRUGIS'] = hru_yr_mean_all.index.get_level_values(0)
hru_yr_mean_all['HRUGIS'] = hru_yr_mean_all['HRUGIS'].apply(lambda x: str(x).zfill(9),1)
hru_yr_mean_all.to_csv(processed_dir + 'hru_yr_mean_all.csv')


In [ ]:
names = ['LULC', 'HRU', 'GIS', 'SUB', 'MGT', 'MO', 'DA', 'YR', 'AREAkm2',
       'PRECIPmm', 'SNOFALLmm', 'SNOMELTmm', 'IRRmm', 'PETmm', 'ETmm',
       'SW_INITmm', 'SW_ENDmm', 'PERCmm', 'GW_RCHGmm', 'DA_RCHGmm', 'REVAPmm',
       'SA_IRRmm', 'DA_IRRmm', 'SA_STmm', 'DA_STmm','SURQ_GENmm','SURQ_CNTmm',
       'TLOSSmm', 'LATQGENmm', 'GW_Qmm', 'WYLDmm']
len(names)

## ArcSWAT TxtInOut RCH Summary for Daily Output

In [ ]:
#output_dir = 'E:/Google Drive/WORK/Round_Valley/Data/GIS/SWAT/Scenarios/short/TxtInOut/'
#output_dir = 'E:/PROJECTS/Round_Valley/SWAT/Scenarios/Long_Run_Mar_2018/TxtInOut/'
#output_dir = 'E:/PROJECTS/Round_Valley/SWAT/Scenarios/Default/TxtInOut/'

output_dir = 'E:/PROJECTS/Round_Valley/SWAT/Scenarios/RevapAndCNadj/TxtInOut/'

In [ ]:
def getPlot(output_dir, subb=1):
    cols = ['RCH','GIS','MO','DA','YR','AREAkm2','FLOW_INcms','FLOW_OUTcms','EVAPcms','TLOSScms']
    rch = pd.read_table(output_dir+'output.rch',sep='\s+',usecols=[1,2,3,4,5,6,7,8,9,10],skiprows=9,names=cols)
    rch['FLOW_OUTacftyr'] = rch['FLOW_OUTcms']*25566.7
    rch['FLOW_OUTcfs'] = rch['FLOW_OUTcms']*35.3147
    byrch = rch.groupby('RCH').median()

    rch['datetime'] = rch[['MO','DA','YR']].apply(lambda x: pd.datetime(x[2],x[0],x[1]),1)
    rch.set_index('datetime',inplace=True)
    valley_out = rch[rch['RCH'] == subb]
    

    valley_out = valley_out['FLOW_OUTcfs'].to_frame()
    return byrch, valley_out, rch
    


In [ ]:
byrch, valley_out, rch = getPlot(output_dir, 56)
discharge = wa.nwis('dv','10140100','sites')
flow_data = discharge.data
flow_data.reset_index(inplace=True)
flow_data.set_index('datetime',inplace=True)
flow_data = flow_data[(flow_data.index >= pd.datetime(2004,1,1))&(flow_data.index < pd.datetime(2017,1,1))]
flow_data = flow_data.resample('1D').mean()
flow_data.interpolate(method='time',inplace=True)

x1 = valley_out.index
y1 = valley_out['FLOW_OUTcfs']
x2 = flow_data.index
y2 = flow_data['value']


byrch, valley_out, rch = getPlot(output_dir, 61)
discharge = wa.nwis('dv','10137500','sites')
flow_data = discharge.data
flow_data.reset_index(inplace=True)
flow_data.set_index('datetime',inplace=True)
flow_data = flow_data[(flow_data.index >= pd.datetime(2010,1,1))&(flow_data.index < pd.datetime(2017,1,1))]
flow_data = flow_data.resample('1D').mean()
flow_data.interpolate(method='time',inplace=True)

x3 = valley_out.index
y3 = valley_out['FLOW_OUTcfs']
x4 = flow_data.index
y4 = flow_data['value']
xl = range(0,1700)


In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 15, 15
rcParams['pdf.fonttype'] = 42
inline_rc = dict(plt.rcParams)
rcParams.update(inline_rc)
rcParams['pdf.fonttype'] = 42

byrch, valley_out, rch = getPlot(output_dir, 7)
drive = 'G:/My Drive/WORK/'
#fileplace =  drive + '/PROJECTS/Round_Valley/'

#fileLocation = fileplace + 'URVH/URVH/'

fileplace = drive + 'Round_Valley/Data/Hydrology_Data/'

lmcfs = pd.read_pickle(fileplace + 'FLOW_W_BASEFLOW.pickle')
DEQmeandly = pd.read_pickle(fileplace+"deqmeandaily.pickle")
DEQ_long_flow = pd.read_pickle(fileplace+"deq_long_flow.pickle")
DEQmeandly.to_clipboard()

x1 = valley_out.index
y1 = valley_out['FLOW_OUTcfs']

plt.plot(x1,y1,label='Modeled (SWAT)')
plt.plot(DEQmeandly.index, DEQmeandly.LowerMain_cfs,label='DEQ Measurements')
plt.plot(lmcfs.index, lmcfs.LowerMain_cfs, label = 'UGS Measurements' )
plt.scatter(DEQ_long_flow.index,DEQ_long_flow.Flow_cfs, color='red',label='Manual Measurements', zorder=10)

plt.yscale('log')
plt.xlim('10/1/2010','10/1/2017')
plt.grid()
plt.ylabel('Lower Main Creek Discharge (cfs)')
plt.legend()

In [ ]:
byrch, valley_out, rch = getPlot(output_dir, 7)


In [ ]:


y = valley_out.loc[lmcfs.index[0]:lmcfs.index[-1],'FLOW_OUTcfs']
x = lmcfs.LowerMain_cfs


lm = pd.concat([x,y],axis=1)
lm.columns = ['meas','model']
lm['logmeas'] = np.log(lm['meas'])
lm['logmodel'] = np.log(lm['model'])

plt.figure()
plt.scatter(x,y)
plt.plot(range(1,200),range(1,200))

g = sns.jointplot("model", "meas", data=lm, kind="reg",
                  color="r", size=7)

In [ ]:
lm = pd.concat([x,y],axis=1)
lm.columns = ['meas','model']
lm['logmeas'] = np.log(lm['meas'])
lm['logmodel'] = np.log(lm['model'])

import seaborn as sns

lmsmall = lm[(lm['logmeas']<2)]
lmbig =  lm[(lm['logmeas']>2)]

g = sns.jointplot("logmodel", "logmeas", data=lm, kind="kde",
                  color="r", size=7)

h = sns.jointplot("logmodel", "logmeas", data=lmsmall, kind="kde",
                  color="b", size=7)
j = sns.jointplot("logmodel", "logmeas", data=lmbig, kind="kde",
                  color="g", size=7)


In [ ]:
byrch, valley_out, rch = getPlot(output_dir, 7)
y = valley_out.loc[DEQmeandly.index[0]:DEQmeandly.index[-1],'FLOW_OUTcfs']
x = DEQmeandly.LowerMain_cfs

plt.figure()
plt.scatter(x,y)



In [ ]:
import matplotlib
matplotlib.rc('pdf', fonttype=42)
rcParams['figure.figsize'] = 15, 15
# Four axes, returned as a 2-d array
fig, ax = plt.subplots(2, 2)

ax[0, 0].plot(x1,y1,label='Modeled (SWAT)')
ax[0, 0].plot(x2,y2,label='Measured (USGS)')
ax[0, 0].set_title('Discharge of Ogden R. below Pinview Res. (USGS site 10140100)')
ax[0, 0].set_ylabel('Discharge (cfs)')
ax[0, 0].set_ylim(0.1,10000)
ax[0, 0].set_yscale('log')
ax[0, 0].grid()

ax[1, 0].plot(x3,y3,label='Modeled (SWAT)')
ax[1, 0].plot(x4,y4,label='Measured (USGS)')
ax[1, 0].set_title('Discharge of South Fk near Huntsville (USGS site 10137500)')
ax[1, 0].set_xlabel('Year')
ax[1, 0].set_ylabel('Discharge (cfs)')
ax[1, 0].set_ylim(0.1,10000)
ax[1, 0].set_yscale('log')
ax[1, 0].grid()

ax[0, 1].scatter(y2,y1,label='modeled and measured correlation')
ax[0, 1].plot(xl,xl,color='red',label='1-1 line')
ax[0, 1].set_ylabel('Modeled Discharge (cfs)')
ax[0, 1].set_ylim(0.1,10000)
ax[0, 1].set_yscale('log')
ax[0, 1].set_xlim(1.0,10000)
ax[0, 1].set_xscale('log')
ax[0, 1].grid()

ax[1, 1].scatter(y4,y3, label='modeled and measured correlation')
ax[1, 1].plot(xl,xl,color='red',label='1-1 line')
ax[1, 1].set_ylabel('Modeled Discharge (cfs)')
ax[1, 1].set_xlabel('Measured Discharge (cfs)')
ax[1, 1].set_ylim(0.1,10000)
ax[1, 1].set_yscale('log')
ax[1, 1].set_xlim(1.0,10000)
ax[1, 1].set_xscale('log')
ax[1, 1].grid()

# Fine-tune figure; hide x ticks for top plots and y ticks for right plots
plt.setp([a.get_xticklabels() for a in ax[0, :]], visible=False)
plt.setp([a.get_yticklabels() for a in ax[:, 1]], visible=False)

h1, l1 = ax[0,0].get_legend_handles_labels()
h2, l2 = ax[0,1].get_legend_handles_labels()

fig.legend(h1+h2, l1+l2, loc=8, bbox_transform=fig.transFigure, ncol=4,scatterpoints = 1 )


fig.tight_layout()
fig.subplots_adjust(bottom=0.10)   

plt.savefig('G:/My Drive/WORK/Ogden Valley/SWAT_figures/USGSvsSWATdischarge.pdf')

In [ ]:
byrch, valley_out, rch = getPlot(output_dir, 7)
valley_out['FLOW_OUTcfs'].groupby(valley_out.index.year).mean()

In [ ]:
valley_out['FLOW_OUTcfd'] = valley_out['FLOW_OUTcfs']*86400
valley_out['FLOW_OUTcfd'].groupby(valley_out.index.year).sum()*0.0000229569 # acre-feet per year

In [ ]:
drive = 'G:/My Drive/WORK/'
fileplace = drive + 'Round_Valley/Data/Hydrology_Data/'
ugs_data = pd.read_pickle(fileplace+'ugs_data_out.pickle')

scenar1 = 'short'
scenar2 = 'Default'
output_dir1 = 'G:/My Drive/WORK/Round_Valley/Data/GIS/SWAT/Scenarios/{:}/TxtInOut/'.format(scenar1)
output_dir2 = 'G:/My Drive/WORK/Round_Valley/Data/GIS/SWAT/Scenarios/{:}/TxtInOut/'.format(scenar2)

byrch1, valley_out1 = getPlot(output_dir1)
byrch2, valley_out2 = getPlot(output_dir2)


ugs_df = ugs_data['LowerMain_cfs'].to_frame()
ugs_df.dropna(inplace=True)
ugs_df = ugs_df.resample('1D').mean()

vo1 = valley_out1[(valley_out1.index >= ugs_df.index[0])&(valley_out1.index <= ugs_df.index[-1])]
vo2 = valley_out2[(valley_out2.index >= ugs_df.index[0])&(valley_out2.index <= ugs_df.index[-1])]
vo1.rename(columns={'FLOW_OUTcfs':scenar1},inplace=True)
vo2.rename(columns={'FLOW_OUTcfs':scenar2},inplace=True)

combined_mc = pd.concat([vo1,vo2,ugs_df],axis=1)
combined_mc.plot()
plt.yscale('log')

In [ ]:
import statsmodels.api as sm
import numpy as np

df = combined_mc[(combined_mc['LowerMain_cfs']>10.0)&(combined_mc['FLOW_OUTcfs']<200)]
y = df.LowerMain_cfs
x = df['FLOW_OUTcfs']

plt.scatter(x, y,color='red')
#valley_out['FLOW_OUTcfs'].plot()
#plt.xlim('1/1/2014','10/1/2017')
#

xa = sm.add_constant(x)
est = sm.RLM(y, xa).fit()
r2 = sm.WLS(y, xa, weights=est.weights).fit().rsquared
slope = est.params[1]
x_prime = np.linspace(np.min(x), np.max(x), 100)[:, np.newaxis]
x_prime = sm.add_constant(x_prime)
y_hat = est.predict(x_prime)

const = est.params[0]
x1 = np.arange(np.min(x), np.max(x), 0.1)
y2 = [i * slope + const for i in x1]
plt.plot(x1,y2)
print(est.summary())
plt.ylim(0,200)
plt.xlim(0,1000)

In [ ]:
combined_mc['mod_adj'] = combined_mc['FLOW_OUTcfs'].apply(lambda x: x*slope + const, 1)

combined_mc.plot()
plt.yscale('log')

## ArcSWAT TxtInOut SUB

In [ ]:
def getsubPlot(output_dir, subb=55):
    cols = ['SUB','GIS', 'MO','DA','YR','AREAkm2','PRECIPmm','SNOMELTmm','PETmm','ETmm','SWmm','PERCmm','SURQmm',
            'GW_Qmm','WYLDmm','LatQmm']
    rch = pd.read_table(output_dir+'output.sub',sep='\s+',usecols=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,22],
                        skiprows=9,names=cols)
    rch['WYLDmcubed'] = rch['AREAkm2']*rch['WYLDmm']*1000.0 # cubic meters
    rch['WYLDacft'] = rch['WYLDmcubed']*0.000810714
    byrch = rch.groupby('SUB').median()

    rch['MDY'] = rch[['MO','DA','YR']].apply(lambda x: pd.datetime(x[2],x[0],x[1]),1)
    rch.set_index('MDY',inplace=True)
    valley_out = rch[rch['SUB'] == subb]
    #valley_out.set_index('datetime',inplace=True)

    #valley_out = valley_out['WYLDmcubed'].to_frame()
    return byrch, valley_out, rch
    


In [ ]:
subfile = "F:/GIS/Ogden_Valley/SWAT_2017_09_11/Scenarios/Model_w_SWATCUP_params_12_2017/TxtInOut/"

In [ ]:
subbysub, sub_valley_out, subrch = getsubPlot(subfile,55)

In [ ]:
rch.reset_index(inplace=True)
rch.set_index('RCH',inplace=True)
subrch.reset_index(inplace=True)
subrch['RCH'] = subrch['SUB']
subrch.set_index('SUB',inplace=True)

In [ ]:
print(len(rch),len(subrch))

In [ ]:
rch_and_sub = pd.concat([rch, subrch],axis=1)
rch_and_sub.reset_index(inplace=True)
rch_and_sub.set_index(['datetime'],inplace=True)
rch_and_sub.rename(columns = {'index':'SUB'},inplace=True)
rch_and_sub.columns

In [ ]:
for col in rch_and_sub.columns:
    if 'mm' in col:
        print(col)
        rch_and_sub[str(col)[:-2]+'acft'] = rch_and_sub[[col,'AREAkm2']].apply(lambda x: x[0]*x[1]*1000.0*0.000810714,1)
rch_and_sub.columns

In [ ]:
rch_and_sub['EVAPcfd'] = rch_and_sub['EVAPcms']*35.3147*86400
rch_and_sub['TLOSScfd'] = rch_and_sub['TLOSScms']*35.3147*86400  
rch_and_sub['FLOW_INcfs'] = rch_and_sub['FLOW_INcms']*35.3147
rch_and_sub['FLOW_INcfd'] = rch_and_sub['FLOW_INcfs']*86400
rch_and_sub['FLOW_OUTcfd'] = rch_and_sub['FLOW_OUTcfs']*86400
rch_and_sub_by_yr = rch_and_sub.groupby(['SUB',rch_and_sub.index.year]).sum()
rch_and_sub_by_yr['FLOW_OUTacft-yr'] = rch_and_sub_by_yr['FLOW_OUTcfd']*0.0000229569
rch_and_sub_by_yr['FLOW_INacft-yr'] = rch_and_sub_by_yr['FLOW_INcfd']*0.0000229569
rch_and_sub_by_yr['TLOSSacft-yr'] = rch_and_sub_by_yr['TLOSScfd']*0.0000229569
rch_and_sub_by_yr['EVAPacft-yr'] = rch_and_sub_by_yr['EVAPcfd']*0.0000229569
rch_and_sub_by_yr = rch_and_sub_by_yr.round(0)

In [ ]:
keepers = []
for col in rch_and_sub_by_yr.columns:
    if 'acft' in col:
        keepers.append(col)

In [ ]:
rch_and_sub_by_wt_yr[keepers].groupby('SUB').mean().to_clipboard()

In [ ]:
rch_and_sub_by_yr[keepers].to_clipboard()

In [ ]:
rch_and_sub_by_wt_yr = rch_and_sub.groupby(['SUB',rch_and_sub.index.shift(-2,freq='m').year]).sum()
rch_and_sub_by_wt_yr['FLOW_OUTacft-yr'] = rch_and_sub_by_wt_yr['FLOW_OUTcfd']*0.0000229569
rch_and_sub_by_wt_yr['FLOW_INacft-yr'] = rch_and_sub_by_wt_yr['FLOW_INcfd']*0.0000229569
rch_and_sub_by_wt_yr['TLOSSacft-yr'] = rch_and_sub_by_wt_yr['TLOSScfd']*0.0000229569
rch_and_sub_by_wt_yr['EVAPacft-yr'] = rch_and_sub_by_wt_yr['EVAPcfd']*0.0000229569
rch_and_sub_by_wt_yr = rch_and_sub_by_wt_yr.round(0)

In [ ]:
rch_and_sub_by_wt_yr[keepers].to_clipboard()

In [ ]:
sub_valley_out['WYLDacft'].groupby(sub_valley_out.index.year).sum()

## SWAT-Cup Flow Out

In [ ]:

fdict = {}
iterlist = ['Iter4','Iter3']


for name in iterlist:
    for shed in range(1,83):
        j = 0
        Flow_filename = "H:/GIS/Ogden_Valley/SWAT_2017_09_11/swatcup/Og20171026.Sufi2.SwatCup/Iterations/{:}/Sufi2.Out/R-FLOW_{:}.txt"
        Flow_file = Flow_filename.format(name,shed)
        for i in range(0,3380,169):
            j += 1
            dname = '{:}-{:02d}-shed{:02d}'.format(name, j, shed)
            fdict[dname] = pd.read_table(Flow_file, sep='\s+',
                                                            skiprows=1+i,nrows=168,
                                                            names=['timestep','flow_cms'])
            fdict[dname]['date'] = pd.date_range('1/1/2003','12/1/2016',freq='MS')
            fdict[dname].set_index(['date'],inplace=True)
            fdict[dname]['watershed'] = shed
            fdict[dname]['model_iter'] = name
            fdict[dname]['flow_acftmo'] = fdict[dname]['flow_cms']*2130.56
iter5 = pd.concat(fdict)

In [ ]:
iter5

In [ ]:
yrmogrp = iter5.groupby([iter5.watershed, iter5.index.get_level_values(1).year,iter5.index.get_level_values(1).month]).median()
yrmogrp = yrmogrp[yrmogrp.index.get_level_values(0) == 56]
yrmogrp.groupby(yrmogrp.index.get_level_values(1)).sum()

In [ ]:
iter5.groupby(iter5.index.get_level_values(1).month).median().sum()

# Climate Raster Zonal Summaries

## SWAT

In [ ]:
arcpy.env.overwriteOutput = True
# Set environment settings
arcpy.env.workspace = "F:/GIS/Ogden_Valley/SWAT_2017_09_11/out_Data.gdb"

# Set local variables
inZoneData = "watersheds_8"
zoneField = "Permanent_Identifier"

for rast in arcpy.ListRasters():
    inValueRaster = rast
    outTable = "zonal_{:}".format(rast)
    try:
        outZSaT = ZonalStatisticsAsTable(inZoneData, zoneField, inValueRaster, outTable, "DATA", "MEAN")
    except:
        pass

In [ ]:
swtdct = getdict("F:/GIS/Ogden_Valley/SWAT_2017_09_11/out_Data.gdb",yrstrt = -4)
swtzone = dictozone(swtdct, yrstrt = -4, dataend = -4, div=1000.0)
swttab = zonal_stack(swtzone, zoneid = 'Permanent_Identifier', dmin=2010, dmax=2016)
SWTdata = get_sum_rows(swttab)

In [ ]:
SWTdata 

In [ ]:
from matplotlib.sankey import Sankey
# http://flothesof.github.io/sankey-tutorial-matplotlib.html
fig = plt.figure()


flows = arcswt_byFk.ix['Middle Fork',['PRECIP_area_adj','ET_area_adj',
                                      'GW_Q_area_adj','SURQ_area_adj',
                                      'PERC_area_adj','delta_SW']]
labs = list(flows.index.values)

sankey = Sankey(unit='ac-ft',scale=0.00001)
sankey.add(flows=list(flows.values),labels = labs,
           orientations=[1, 1, 0, 1,-1,-1], facecolor='blue')
sankey.finish()
plt.legend(loc='best')
# Notice that only one connection is specified, but the systems form a
# circuit since: (1) the lengths of the paths are justified and (2) the
# orientation and ordering of the flows is mirrored.

### SWAT Zonal

In [6]:
arcpy.env.overwriteOutput = True
# Set environment settings
arcpy.env.workspace = "E:/PROJECTS/Round_Valley/UBM/UBM_RESAMP.gdb"

# Set local variables
inZoneData = {"E:/PROJECTS/Round_Valley/UBM_Zonal.gdb/GeoZones":["name","geo"],
              "E:/PROJECTS/Round_Valley/UBM_Zonal.gdb/HUC_Zones":["HUC_12","huc"]}

zoneField = "Permanent_Identifier"

for rast in arcpy.ListRasters():
    for key,value in inZoneData.items():
        outTable = "E:/PROJECTS/Round_Valley/UBM_Zonal2.gdb/{:}_{:}".format(value[1],rast)
        outZSaT = ZonalStatisticsAsTable(key, value[0], rast, outTable, "DATA", "MEAN")
        print(rast, value)

rec200401 ['name', 'geo']
rec200401 ['HUC_12', 'huc']
run200401 ['name', 'geo']
run200401 ['HUC_12', 'huc']
aet200401 ['name', 'geo']
aet200401 ['HUC_12', 'huc']
asw200401 ['name', 'geo']
asw200401 ['HUC_12', 'huc']
rec200402 ['name', 'geo']
rec200402 ['HUC_12', 'huc']
run200402 ['name', 'geo']
run200402 ['HUC_12', 'huc']
aet200402 ['name', 'geo']
aet200402 ['HUC_12', 'huc']
asw200402 ['name', 'geo']
asw200402 ['HUC_12', 'huc']
rec200403 ['name', 'geo']
rec200403 ['HUC_12', 'huc']
run200403 ['name', 'geo']
run200403 ['HUC_12', 'huc']
aet200403 ['name', 'geo']
aet200403 ['HUC_12', 'huc']
asw200403 ['name', 'geo']
asw200403 ['HUC_12', 'huc']
rec200404 ['name', 'geo']
rec200404 ['HUC_12', 'huc']
run200404 ['name', 'geo']
run200404 ['HUC_12', 'huc']
aet200404 ['name', 'geo']
aet200404 ['HUC_12', 'huc']
asw200404 ['name', 'geo']
asw200404 ['HUC_12', 'huc']
rec200405 ['name', 'geo']
rec200405 ['HUC_12', 'huc']
run200405 ['name', 'geo']
run200405 ['HUC_12', 'huc']
aet200405 ['name', 'geo']
ae

rec200703 ['name', 'geo']
rec200703 ['HUC_12', 'huc']
run200703 ['name', 'geo']
run200703 ['HUC_12', 'huc']
aet200703 ['name', 'geo']
aet200703 ['HUC_12', 'huc']
asw200703 ['name', 'geo']
asw200703 ['HUC_12', 'huc']
rec200704 ['name', 'geo']
rec200704 ['HUC_12', 'huc']
run200704 ['name', 'geo']
run200704 ['HUC_12', 'huc']
aet200704 ['name', 'geo']
aet200704 ['HUC_12', 'huc']
asw200704 ['name', 'geo']
asw200704 ['HUC_12', 'huc']
rec200705 ['name', 'geo']
rec200705 ['HUC_12', 'huc']
run200705 ['name', 'geo']
run200705 ['HUC_12', 'huc']
aet200705 ['name', 'geo']
aet200705 ['HUC_12', 'huc']
asw200705 ['name', 'geo']
asw200705 ['HUC_12', 'huc']
rec200706 ['name', 'geo']
rec200706 ['HUC_12', 'huc']
run200706 ['name', 'geo']
run200706 ['HUC_12', 'huc']
aet200706 ['name', 'geo']
aet200706 ['HUC_12', 'huc']
asw200706 ['name', 'geo']
asw200706 ['HUC_12', 'huc']
rec200707 ['name', 'geo']
rec200707 ['HUC_12', 'huc']
run200707 ['name', 'geo']
run200707 ['HUC_12', 'huc']
aet200707 ['name', 'geo']
ae

rec201005 ['name', 'geo']
rec201005 ['HUC_12', 'huc']
run201005 ['name', 'geo']
run201005 ['HUC_12', 'huc']
aet201005 ['name', 'geo']
aet201005 ['HUC_12', 'huc']
asw201005 ['name', 'geo']
asw201005 ['HUC_12', 'huc']
rec201006 ['name', 'geo']
rec201006 ['HUC_12', 'huc']
run201006 ['name', 'geo']
run201006 ['HUC_12', 'huc']
aet201006 ['name', 'geo']
aet201006 ['HUC_12', 'huc']
asw201006 ['name', 'geo']
asw201006 ['HUC_12', 'huc']
rec201007 ['name', 'geo']
rec201007 ['HUC_12', 'huc']
run201007 ['name', 'geo']
run201007 ['HUC_12', 'huc']
aet201007 ['name', 'geo']
aet201007 ['HUC_12', 'huc']
asw201007 ['name', 'geo']
asw201007 ['HUC_12', 'huc']
rec201008 ['name', 'geo']
rec201008 ['HUC_12', 'huc']
run201008 ['name', 'geo']
run201008 ['HUC_12', 'huc']
aet201008 ['name', 'geo']
aet201008 ['HUC_12', 'huc']
asw201008 ['name', 'geo']
asw201008 ['HUC_12', 'huc']
rec201009 ['name', 'geo']
rec201009 ['HUC_12', 'huc']
run201009 ['name', 'geo']
run201009 ['HUC_12', 'huc']
aet201009 ['name', 'geo']
ae

rec201307 ['name', 'geo']
rec201307 ['HUC_12', 'huc']
run201307 ['name', 'geo']
run201307 ['HUC_12', 'huc']
aet201307 ['name', 'geo']
aet201307 ['HUC_12', 'huc']
asw201307 ['name', 'geo']
asw201307 ['HUC_12', 'huc']
rec201308 ['name', 'geo']
rec201308 ['HUC_12', 'huc']
run201308 ['name', 'geo']
run201308 ['HUC_12', 'huc']
aet201308 ['name', 'geo']
aet201308 ['HUC_12', 'huc']
asw201308 ['name', 'geo']
asw201308 ['HUC_12', 'huc']
rec201309 ['name', 'geo']
rec201309 ['HUC_12', 'huc']
run201309 ['name', 'geo']
run201309 ['HUC_12', 'huc']
aet201309 ['name', 'geo']
aet201309 ['HUC_12', 'huc']
asw201309 ['name', 'geo']
asw201309 ['HUC_12', 'huc']
rec201310 ['name', 'geo']
rec201310 ['HUC_12', 'huc']
run201310 ['name', 'geo']
run201310 ['HUC_12', 'huc']
aet201310 ['name', 'geo']
aet201310 ['HUC_12', 'huc']
asw201310 ['name', 'geo']
asw201310 ['HUC_12', 'huc']
rec201311 ['name', 'geo']
rec201311 ['HUC_12', 'huc']
run201311 ['name', 'geo']
run201311 ['HUC_12', 'huc']
aet201311 ['name', 'geo']
ae

In [ ]:
arcpy.env.overwriteOutput = True
# Set environment settings
arcpy.env.workspace = "E:/GIS/UBM/Available_Water.gdb"

# Set local variables
inZoneData = {"E:/PROJECTS/Round_Valley/UBM_Zonal.gdb/GeoZones":["name","geo"],
              "E:/PROJECTS/Round_Valley/UBM_Zonal.gdb/HUC_Zones":["HUC_12","huc"]}

for rast in arcpy.ListRasters():
    for key,value in inZoneData.items():
        outTable = "E:/PROJECTS/Round_Valley/AVWT_Zonal.gdb/{:}_{:}".format(value[1],rast)
        outZSaT = ZonalStatisticsAsTable(key, value[0], rast, outTable, "DATA", "MEAN")


In [7]:
def calcvols(searchStr='*', stat='MEAN', mult = 1.0):
    tables = arcpy.ListTables(searchStr)    
    f = {}
    for table in tables:
        fields = arcpy.ListFields(table)
        #for table in prism_tables:
        fieldlist = [field.name for field in fields]
        f[table] = pd.DataFrame(arcpy.da.TableToNumPyArray(table,fieldlist))
    g = pd.concat(f)
    g.reset_index(inplace=True)

    g['raster_name'] = g['level_0'].apply(lambda x: str(x),1)
    g['datav'] = g['level_0'].apply(lambda x: str(x)[:4],1)
    g['YearMonth'] = g['level_0'].apply(lambda x: str(x)[-6:],1)
    g['Year'] = g['level_0'].apply(lambda x: str(x)[-6:-2],1)
    g['Month'] = g['level_0'].apply(lambda x: str(x)[-2:],1)

    g.drop(['level_0','level_1','OBJECTID','ZONE_CODE'],axis=1,inplace=True)
    g['SOURCE'] = "SNODAS"

    g['volume_m_cubed'] = g[[stat,'AREA']].apply(lambda x: round(x[0]*x[1]*mult,0),1)
    g['volume_acft'] = g['volume_m_cubed'].apply(lambda x: round(x*0.000810714,0),1)

    g['date'] = g.apply(lambda x: pd.to_datetime(x.YearMonth,errors='coerce',format='%Y%m'),1)
    return g

In [8]:
engineroute = "G:/My Drive/WORK/Round_Valley/Data/"

sys.path.append(engineroute)
import enginegetter
engine = enginegetter.getEngine()

In [9]:
arcpy.env.workspace ="E:/PROJECTS/Round_Valley/UBM_Zonal2.gdb"
tabname = 'roundvalley5'
zones = ['geo','huc']
datasets = ['aet','asw','rec','run']
for zone in zones:
    for data in datasets:
        g = calcvols(searchStr='{:}_{:}*'.format(zone, data))
        if "HUC_12" in g.columns:
            g.rename(columns={"HUC_12":"name"},inplace=True)
        g.to_sql(con=engine, name=tabname, if_exists='append', index=False)
        print(zone, data)

geo aet
geo asw
geo rec
geo run
huc aet
huc asw
huc rec
huc run


## SNODAS

In [ ]:
# Set local variables
arcpy.env.overwriteOutput = True
# Set environment settings
arcpy.env.workspace = "F:/GIS/Ogden_Valley/New Folder/WB_SNODAS.gdb"


inZoneData = "F:/GIS/Ogden_Valley/SWAT_2017_09_11/out_Data.gdb/watersheds_8"
zoneField = "Permanent_Identifier"
for rast in arcpy.ListRasters():

    outTable = "F:/GIS/Ogden_Valley/New Folder/og_SNOTEL_zonal.gdb/zonal_{:}".format(rast)
    outZSaT = ZonalStatisticsAsTable(inZoneData, zoneField, rast, outTable, "DATA", "MEAN")



In [ ]:
snodct = getdict("F:/GIS/Ogden_Valley/New Folder/og_SNOTEL_zonal.gdb",yrstrt = -9)
snozone = dictozone(snodct, yrstrt = -9, dataend = 4)
snotab = zonal_stack(snozone, zoneid = 'Permanent_Identifier', dmin=2010, dmax=2015)
SNOdata = get_sum_rows(snotab)

## PRISM

In [ ]:
#arcpy.env.workspace = "E:/GIS/PRISM_raw"
arcpy.env.workspace = "E:/Users/paulinkenbrandt/Downloads/PRISM_ppt_30yr_normal_800mM2_all_bil"
#project = "USA Contiguous Albers Equal Area Conic USGS version"
sr = arcpy.SpatialReference(102039)

cellSize = 150

arcpy.env.overwriteOutput = True
arcpy.env.mask = "E:/GIS/PRISM.gdb/UT_HUCS"
# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")

for rast in arcpy.ListRasters():
    arcpy.ProjectRaster_management(rast,"in_memory/projPRSM",sr,"CUBIC")
    # Execute ExtractByMask
    outExtractByMask = arcpy.sa.ExtractByMask("in_memory/projPRSM", "E:/GIS/PRISM.gdb/UT_HUCS")

    # Save the output 
    outExtractByMask.save("E:/GIS/PRISM.gdb/{:}".format(rast[:-4]))

    print(rast[:-4])

In [ ]:
# Set local variables
arcpy.env.overwriteOutput = True
# Set environment settings
arcpy.env.workspace = "E:\GIS\PRISM.gdb"


inZoneData = "F:/GIS/Ogden_Valley/SWAT_2017_09_11/out_Data.gdb/watersheds_8"
zoneField = "Permanent_Identifier"
for rast in arcpy.ListRasters():
    if rast[1:5] in [str(i) for i in range(2000,2018)]:
        outTable = "F:/GIS/Ogden_Valley/New Folder/og_PRISM_zonal.gdb/zonal_{:}".format(rast)
        outZSaT = ZonalStatisticsAsTable(inZoneData, zoneField, rast, outTable, "DATA", "MEAN")


In [ ]:
prsdct = getdict("F:/GIS/Ogden_Valley/New Folder/og_PRISM_zonal.gdb",yrstrt = -6)



In [ ]:
prszone = dictozone(prsdct, yrstrt = -6, dataend = 1,div=1000.0)
prstab = zonal_stack(prszone, zoneid = 'Permanent_Identifier', dmin=2010, dmax=2016)
PRSdata = get_sum_rows(prstab)

## UBM

In [ ]:
# Set local variables
arcpy.env.overwriteOutput = True
# Set environment settings
arcpy.env.workspace = "F:/GIS/Ogden_Valley/New Folder/Fixed_Results.gdb"


inZoneData = "F:/GIS/Ogden_Valley/SWAT_2017_09_11/out_Data.gdb/watersheds_8"
zoneField = "Permanent_Identifier"

for rast in arcpy.ListRasters():
    if rast[-6:-2] in [str(i) for i in range(2000,2018)]:
        outTable = "F:/GIS/Ogden_Valley/New Folder/og_UBM_zonal.gdb/zonal_{:}".format(rast)
        outZSaT = ZonalStatisticsAsTable(inZoneData, zoneField, rast, outTable, "DATA", "MEAN")


In [ ]:
ubmdct = getdict("F:/GIS/Ogden_Valley/New Folder/og_UBM_zonal.gdb")
ubmzone = dictozone(ubmdct)


In [ ]:
ubmtab = zonal_stack(ubmzone)
UBMdata = get_sum_rows(ubmtab)

## Combine Data

In [ ]:
writer = pd.ExcelWriter(processed_dir+'Summary_zonal1.xlsx')
PRSdata.to_excel(writer,'PRISM')
SWTdata.to_excel(writer,'SWAT')
UBMdata.to_excel(writer,'UBM')
SNOdata.to_excel(writer,'SNODAS')
writer.save()

In [ ]:
years = ['2010','2011','2012','2013','2014','2015','2016']
SWTdata.loc['precip-ET'] = 0
SWTdata.loc['precip-ET'].loc[:,years] = SWTdata.loc['PRECIPmm'].loc[:,years] - SWTdata.loc['ETmm'].loc[:,years]


In [ ]:
SWTdata.reset_index(inplace=True)

In [ ]:
SWTdata['data'] = SWTdata['index'].map(lambda x: x[0])
SWTdata['area'] =  SWTdata['index'].map(lambda x: x[1])
SWTdata.drop(['index'],axis=1,inplace=True)

In [ ]:
SWTdata.set_index(['data','area'],inplace=True)

In [ ]:
labels = list(set(SWTdata.index.get_level_values(0).values))
labels

In [ ]:
areas = ['WCBRX','SFVF','SFBRX','Pineview','NFVF','NFBRX','MFVF','MFBRX']
peares = ['WCBRX','SFBRX','Pineview','NFBRX','MFVF','MFBRX']
years = [str(i) for i in range (2010,2015)]

In [ ]:
PRSdata.loc['P'].loc[areas].index.values
areas

In [ ]:
print(PRSdata.loc['P'].loc[areas].index.values, SWTdata.loc['PRECIPmm'].loc[areas].index.values)
PRSdata.loc['P'].loc[areas,years]

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

def olsplot(x,y,xname, ax):

#x = zsum.loc['WYLD'].loc['south_fork',years]
#y = yearly_q.loc[[int(i) for i in years],'acftyr']
    #x.index = x.index.map(int)
    #y.index = y.index.map(int)
    df = pd.DataFrame({'x':x,'y':y})


    #X = sm.add_constant(x.values)
    results = smf.ols('y~x', data=df).fit()

    print(results.summary())

    modx = df[x]
    mody = [results.params.Intercept + i* results.params.x for i in modx]
    ax.plot(modx,mody, label = 'best fit line\nr-sqaured = {:.2f}'.format(results.rsquared))
    ax.scatter(x,y, label = 'data')
    ax.plot(range(0,300000),range(0,300000), label = '1-1 line')
    ax.set_ylim(5000,200000)
    ax.set_xlim(5000,200000)
    ax.grid()
    
    return modx, mody


#plt.ylabel('SWAT modeled Water Yeild (acft/yr)')
#plt.xlabel('USGS Measured Flow (acft/yr)')
#plt.legend(loc=2)
#plt.savefig()

In [ ]:
pd.DataFrame({'x':xdat,'y':ydat})

In [ ]:
fig, ax = plt.subplots(2, 2)


norm = mpl.colors.Normalize(vmin=0,vmax=len(areas))
i = 0
for area in areas:

    ax[0, 0].scatter(SNOdata.loc['SNML'].loc[area,years],
                SWTdata.loc['SNOMELTmm'].loc[area,years], color=plt.cm.cool(norm(i)), label=str(area))
    i += 1


x = range(0,500000,20)
y = [1*i for i in x]
ax[0, 0].plot(x,y,'-.',label='1-1 line')
ax[0, 0].set_ylabel('SWAT Snowmelt (ac-ft/yr)')
ax[0, 0].set_xlabel('SNODAS Snowmelt (ac-ft/yr)')

ax[0, 0].grid()
ax[0, 0].set_xlim(100,500000)
ax[0, 0].set_ylim(100,500000)
ax[0, 0].set_xscale('log')
ax[0, 0].set_yscale('log')





i = 0
for area in areas:

    ax[0, 1].scatter(UBMdata.loc['aet2'].loc[area,years],
                SWTdata.loc['ETmm'].loc[area,years], color=plt.cm.cool(norm(i)), label=str(area))
    i += 1


ax[0, 1].plot(x,y,'-.',label='1-1 line')
ax[0, 1].set_ylabel('SWAT Evapotranspiration (ac-ft/yr)')
ax[0, 1].set_xlabel('UBM Evapotranspiration (ac-ft/yr)')

ax[0, 1].grid()
ax[0, 1].set_xlim(100,500000)
ax[0, 1].set_ylim(100,500000)
ax[0, 1].set_xscale('log')
ax[0, 1].set_yscale('log')



i = 0
for area in areas:

    ax[1, 0].scatter(UBMdata.loc['rec2'].loc[area,years],
                SWTdata.loc['PERCmm'].loc[area,years], color=plt.cm.cool(norm(i)), label=str(area))
    i += 1
    
    

ax[1, 0].plot(x,y,'-.',label='1-1 line')
ax[1, 0].set_ylabel('SWAT Percolation (ac-ft/yr)')
ax[1, 0].set_xlabel('UBM Recharge (ac-ft/yr)')
ax[1, 0].grid()
ax[1, 0].set_xlim(100,500000)
ax[1, 0].set_ylim(100,500000)
ax[1, 0].set_xscale('log')
ax[1, 0].set_yscale('log')



i = 0
for area in areas:
    
    if area not in ['SFVF','NFVF']:
        ax[1, 1].scatter(PRSdata.loc['P'].loc[area,years], SWTdata.loc['PRECIPmm'].loc[area,years],
                    color=plt.cm.cool(norm(i)), label=str(area))
    i += 1
    

ax[1, 1].plot(x,y,'-.',label='1-1 line')
ax[1, 1].set_ylabel('SWAT Precipitation (ac-ft/yr)')
ax[1, 1].set_xlabel('PRISM Precipitation (ac-ft/yr)')

ax[1, 1].grid()
ax[1, 1].set_xlim(100,500000)
ax[1, 1].set_ylim(100,500000)
ax[1, 1].set_xscale('log')
ax[1, 1].set_yscale('log')

# Fine-tune figure; hide x ticks for top plots and y ticks for right plots
plt.setp([a.get_xticklabels() for a in ax[0, :]], visible=False)
plt.setp([a.get_yticklabels() for a in ax[:, 1]], visible=False)

h1, l1 = ax[0,0].get_legend_handles_labels()
h2, l2 = ax[0,1].get_legend_handles_labels()

fig.legend(h1, l1, loc=8, bbox_transform=fig.transFigure, ncol=4, scatterpoints = 1 )


fig.tight_layout()
fig.subplots_adjust(bottom=0.10)   

plt.savefig('G:/My Drive/WORK/Ogden Valley/SWAT_figures/USGSvsOtherModels.pdf')

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)


byrch, valley_out = getPlot(output_dir, 38)
discharge = wa.nwis('dv','10137780','sites')
flow_data = discharge.data
flow_data.reset_index(inplace=True)
flow_data.set_index('datetime',inplace=True)
#flow_data = flow_data[(flow_data.index >= pd.datetime(2010,1,1))&(flow_data.index < pd.datetime(2017,1,1))]
flow_data = flow_data.resample('1D').mean()
flow_data.interpolate(method='time',inplace=True)

dailyusgsflow = flow_data.groupby(flow_data.index.dayofyear).mean()
dailyswatflow = valley_out.groupby(valley_out.index.dayofyear).mean()

x = dailyusgsflow.index
y1 = dailyusgsflow.value
y2 = dailyswatflow.FLOW_OUTcfs

usgs_first = discharge.data.first_valid_index()
usgs_last = discharge.data.last_valid_index()
swat_first = valley_out.first_valid_index()
swat_last = valley_out.last_valid_index()


ax[0].plot(x,y1,label = 'Average USGS-Measured Runoff {:%m-%d-%Y} to {:%m-%d-%Y}'.format(usgs_first,usgs_last))
ax[0].plot(x,y2,label =  'Average SWAT-Modeled Runoff {:%m-%d-%Y} to {:%m-%d-%Y}'.format(swat_first,swat_last))
ax[0].set_title('Middle Fork (USGS site 10137780)')
ax[0].set_ylabel('Discharge (cfs)')
ax[0].set_xlim(0,365)

dtrng = pd.date_range('1/1/2016','12/31/2016',freq='1M')
ticklocs = [0] + [float(d.strftime('%j')) for d in dtrng][:-1]
datelabels = [d.strftime('%b') for d in dtrng]

ax[0].set_xticks(ticklocs, datelabels)
ax[0].grid()

byrch, valley_out = getPlot(output_dir, 44)
discharge = wa.nwis('dv','10137700','sites')
flow_data = discharge.data
flow_data.reset_index(inplace=True)
flow_data.set_index('datetime',inplace=True)
#flow_data = flow_data[(flow_data.index >= pd.datetime(2010,1,1))&(flow_data.index < pd.datetime(2017,1,1))]
flow_data = flow_data.resample('1D').mean()
flow_data.interpolate(method='time',inplace=True)

dailyusgsflow = flow_data.groupby(flow_data.index.dayofyear).mean()
dailyswatflow = valley_out.groupby(valley_out.index.dayofyear).mean()

x = dailyusgsflow.index
y1 = dailyusgsflow.value
y2 = dailyswatflow.FLOW_OUTcfs

usgs_first = discharge.data.first_valid_index()
usgs_last = discharge.data.last_valid_index()
swat_first = valley_out.first_valid_index()
swat_last = valley_out.last_valid_index()


ax[1].plot(x,y1,label = 'Average USGS-Measured Runoff {:%m-%d-%Y} to {:%m-%d-%Y}'.format(usgs_first,usgs_last))
ax[1].plot(x,y2,label =  'Average SWAT-Modeled Runoff {:%m-%d-%Y} to {:%m-%d-%Y}'.format(swat_first,swat_last))

ax[1].set_ylabel('Discharge (cfs)')
ax[1].set_xlabel('month')
ax[1].set_xlim(0,365)
ax[1].set_title('North Fork (USGS site 10137700)')
dtrng = pd.date_range('1/1/2016','12/31/2016',freq='1M')
ticklocs = [0] + [float(d.strftime('%j')) for d in dtrng][:-1]
datelabels = [d.strftime('%b') for d in dtrng]
ax[1].set_xticks(ticklocs, datelabels)
ax[1].grid()

h2, l2 = ax[0].get_legend_handles_labels()

fig.legend(h2, l2, loc=8, bbox_transform=fig.transFigure, ncol=4, scatterpoints = 1 )

plt.savefig('G:/My Drive/WORK/Ogden Valley/SWAT_figures/midfork_comp.pdf')

In [ ]:
import seaborn as sns
sns.set(style="darkgrid", color_codes=True)

byrch, valley_out = getPlot(output_dir, 44)
discharge = wa.nwis('dv','10137700','sites')
flow_data = discharge.data
flow_data.reset_index(inplace=True)
flow_data.set_index('datetime',inplace=True)
#flow_data = flow_data[(flow_data.index >= pd.datetime(2010,1,1))&(flow_data.index < pd.datetime(2017,1,1))]
flow_data = flow_data.resample('1D').mean()
flow_data.interpolate(method='time',inplace=True)

dailyusgsflow = flow_data.groupby(flow_data.index.dayofyear).mean()
dailyswatflow = valley_out.groupby(valley_out.index.dayofyear).mean()

x = dailyusgsflow.index
y1 = dailyusgsflow.value
y2 = dailyswatflow.FLOW_OUTcfs

mybins=np.logspace(0,np.log(500),100)

g = sns.jointplot(y1, y2,  kind="reg",
                  xlim=(0.001, 500), ylim=(0.001, 500), color="r", size=7)
#g.plot_marginals(sns.distplot, hist=True, kde=True, color='blue',bins=mybins)

ax = g.ax_joint
ax.set_xscale('log')
ax.set_yscale('log')
g.ax_marg_x.set_xscale('log')
g.ax_marg_y.set_yscale('log')

In [ ]:
byrch, valley_out = getPlot(output_dir, 38)
discharge = wa.nwis('dv','10137780','sites')
flow_data = discharge.data
flow_data.reset_index(inplace=True)
flow_data.set_index('datetime',inplace=True)
#flow_data = flow_data[(flow_data.index >= pd.datetime(2010,1,1))&(flow_data.index < pd.datetime(2017,1,1))]
flow_data = flow_data.resample('1D').mean()
flow_data.interpolate(method='time',inplace=True)

dailyusgsflow = flow_data.groupby(flow_data.index.dayofyear).mean()
dailyswatflow = valley_out.groupby(valley_out.index.dayofyear).mean()

x = dailyusgsflow.index
y1 = dailyusgsflow.value
y2 = dailyswatflow.FLOW_OUTcfs

mybins=np.logspace(0,np.log(500),100)

g = sns.jointplot(y1, y2,  kind="reg",
                  xlim=(0, 500), ylim=(0, 500), color="r", size=7)
#g.plot_marginals(sns.distplot, hist=True, kde=True, color='blue',bins=mybins)



In [ ]:
plt.scatter(y1,y2)
x2 = range(0,350)
plt.plot(x2,x2)
plt.grid()
plt.xlim(0,350)
plt.ylim(0,350)


df = pd.concat([y1,y2],axis=1)


#X = sm.add_constant(x.values)
results = smf.ols('value~FLOW_OUTcfs', data=df).fit()
parms = results.params
print(results.summary())

modx = df['FLOW_OUTcfs']
mody = [parms.Intercept + i* parms.FLOW_OUTcfs for i in modx]
plt.plot(modx,mody, label = 'best fit line\nr-sqaured = {:.2f}'.format(results.rsquared))
#plt.scatter(x,y, label = 'data')



In [ ]:

x1 = valley_out.index
y1 = valley_out['FLOW_OUTcfs']
x2 = flow_data.index
y2 = flow_data['value']

plt.figure()
plt.plot(x1,y1,label='SWAT')
plt.plot(x2,y2,label='USGS')
plt.title('Discharge at USGS site 10137500')
plt.legend()
plt.ylabel('Discharge (cfs)')
#plt.xlim('1/1/2014','10/1/2017')
#plt.ylim(0,1000)
plt.yscale('log')
plt.savefig(processed_dir+'USGSandSWATdischargeBasinOut10137500.pdf')

plt.figure()
plt.scatter(y2,y1)
x3 = range(0,1700)
y3 = x3
plt.plot(x3,x3,color='red')
plt.xlim(0,1800)
plt.ylim(0,1800)
plt.grid()
plt.title('Discharge at USGS site 10137500')
plt.ylabel('SWAT Discharge (cfs)')
plt.xlabel('USGS Discharge (cfs)')
plt.savefig(processed_dir+'USGSvsSWATdischargeBasinOut10137500.pdf')